# Imports

In [4]:
import math
import numpy as np
import pandas as pd
import copy

In [5]:
# from google.colab import drive
# drive.mount('/content/drive')

In [6]:
data_nov = pd.read_csv('2019-Nov.csv')
data_oct = pd.read_csv('2019-Oct.csv')
data_dec = pd.read_csv('2019-Dec.csv')
data_jan = pd.read_csv('2020-Jan.csv')
data_feb = pd.read_csv('2020-Feb.csv')

# Defining functions and generating preferences

In [12]:
count = 1
def enumerate_views(df, row):
    global count
    if row['event_type'] != 'view':
        if row['event_type'] == 'purchase':
            row['was_bought'] = 1
            row['position'] = count - 1
        return row
    if row['index'] != 0: 
          if row['user_session'] != df.iloc[row['index'] - 1, 5]:
              count = 1
              # row['position'] = count
          row['position'] = count
          count += 1
    # if row['position'] != 1:
    #   print(row['position'])
    return row
    # else:
    #   row['position'] = -1

In [13]:
def generate_prefences_month(data_month):
    data_month.dropna(subset=['user_session'], inplace=True)
    data_month['time'] = pd.to_datetime(data_month['event_time'].apply(lambda x: x.replace('UTC', '')))
    data_month.drop(['event_time'], inplace=True, axis=1)
    data_month = data_month[data_month.event_type.isin(['view', 'purchase'])]

    data_month_sorted = data_month.sort_values(by=['user_id', 'user_session', 'time'])
    data_month_sorted.drop(['category_id', 'category_code', 'brand'], axis=1, inplace=True)
    data_month_sorted.reset_index(inplace=True)
    data_month_sorted['position'] = 1
    data_month_sorted['was_bought'] = 0
    data_month_sorted.drop('index', axis=1, inplace=True)
    data_month_sorted.reset_index(inplace=True)
    data_month_sorted.rename(columns={'level_0' : 'index'}, inplace=True)

    global count
    count = 1
    data_month_sorted = data_month_sorted.apply(lambda x: enumerate_views(data_month_sorted, x), axis=1)

    filtered_products = data_month_sorted.loc[data_month_sorted['was_bought'] == 1]['product_id'].to_list()
    data_month_filtered = data_month_sorted.loc[data_month_sorted['product_id'].isin(filtered_products)]

    data_month_firstpos = data_month_filtered.loc[data_month_filtered['position'] < 9]
    data_month_firstpos.drop(['price', 'index'], axis=1, inplace=True)

    grouped_month = data_month_firstpos.groupby(['user_id','user_session', 'product_id', 'position']).sum()
    session_avg_month = grouped_month.groupby(['user_id', 'product_id', 'position']).mean()

    return session_avg_month


preferences_nov = generate_prefences_month(data_nov)
preferences_nov

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


was_bought
user_id   product_id position            
1120748   5240       1                0.0
1458813   5711087    1                0.0
          5711145    1                0.0
          5727766    7                0.0
          5800349    2                0.0
...                                   ...
579968623 5892179    3                0.0
                     5                0.0
          5900651    4                0.0
579968937 5790563    1                0.0
579969854 5880201    1                0.0

[1579034 rows x 1 columns]

In [14]:
preferences_oct = generate_prefences_month(data_oct)
preferences_oct

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


was_bought
user_id   product_id position            
4103071   5563536    4                0.0
          5670334    2                0.0
          5735272    3                0.0
          5784982    1                0.0
                     6                0.0
...                                   ...
566279638 5728072    1                0.0
566279883 5618276    1                0.0
566279996 5700137    1                0.0
566280278 5809910    1                0.0
566280798 59996      1                0.0

[1395974 rows x 1 columns]

In [15]:
preferences_dec = generate_prefences_month(data_dec)
preferences_dec

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


was_bought
user_id   product_id position            
2963072   5707747    2                0.0
                     4                0.0
                     6                0.0
                     7                0.0
          5746011    1                0.0
...                                   ...
595413843 5696330    1                0.0
595413976 5745711    1                0.0
595414210 5886338    1                0.0
595414257 5812934    1                0.0
595414541 59975      1                0.0

[1289088 rows x 1 columns]

In [16]:
preferences_jan = generate_prefences_month(data_jan)
preferences_jan

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


was_bought
user_id   product_id position            
465496    5769989    1                0.0
          5865526    1                0.0
4661182   5809910    1                0.0
                     4                0.0
          5809911    5                0.0
...                                   ...
608821382 5829707    1                0.0
                     2                0.0
608821614 5887255    1                0.0
608821904 5751383    1                0.0
608822072 5751383    1                0.0

[1521925 rows x 1 columns]

In [17]:
preferences_feb = generate_prefences_month(data_feb)
preferences_feb

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


was_bought
user_id   product_id position            
3571443   5814877    2                0.0
          5913642    1                0.0
4230352   4210       1                0.0
4661182   4185       1                0.0
          5809910    2                0.0
...                                   ...
622090043 5809879    1                0.0
          5850628    2                0.0
622090052 5688691    1                0.0
622090098 5650609    1                0.0
622090237 5754853    1                0.0

[1445409 rows x 1 columns]

In [18]:
preferences_oct.reset_index(inplace=True)
preferences_nov.reset_index(inplace=True)
preferences_dec.reset_index(inplace=True)
preferences_jan.reset_index(inplace=True)
preferences_feb.reset_index(inplace=True)

In [19]:
preferences_all_unsorted = pd.concat([preferences_oct, preferences_nov, preferences_dec, preferences_jan, preferences_feb], axis=0, ignore_index=True).groupby(['user_id', 'product_id', 'position']).mean()
preferences_all_unsorted.reset_index(inplace=True)

In [20]:
preferences_all = preferences_all_unsorted.sort_values(by=['user_id', 'product_id', 'position'])
preferences_all

,user_id,product_id,position,was_bought
0,465496,5769989,1,0.0
1,465496,5865526,1,0.0
2,1120748,5240,1,0.0
3,1458813,5711087,1,0.0
4,1458813,5711145,1,0.0
...,...,...,...,...
7105371,622090043,5809879,1,0.0
7105372,622090043,5850628,2,0.0
7105373,622090052,5688691,1,0.0
7105374,622090098,5650609,1,0.0


# Building arrays of preferences

In [21]:
for i in range(1, 9):
    preferences_all[f'p_array{i}'] = (i == preferences_all['position']) * preferences_all['was_bought']

In [22]:
preferences = preferences_all.drop(['position', 'was_bought'], axis=1).groupby(['user_id', 'product_id']).sum()
preferences

p_array1  p_array2  p_array3  p_array4  p_array5  \
user_id   product_id                                                     
465496    5769989          0.0       0.0       0.0       0.0       0.0   
          5865526          0.0       0.0       0.0       0.0       0.0   
1120748   5240             0.0       0.0       0.0       0.0       0.0   
1458813   5711087          0.0       0.0       0.0       0.0       0.0   
          5711145          0.0       0.0       0.0       0.0       0.0   
...                        ...       ...       ...       ...       ...   
622090043 5809879          0.0       0.0       0.0       0.0       0.0   
          5850628          0.0       0.0       0.0       0.0       0.0   
622090052 5688691          0.0       0.0       0.0       0.0       0.0   
622090098 5650609          0.0       0.0       0.0       0.0       0.0   
622090237 5754853          0.0       0.0       0.0       0.0       0.0   

                      p_array6  p_array7  p_array8  
user_id   product_id                                
465496    5769989          0.0       0.0       0.0  
          5865526          0.0       0.0       0.0  
1120748   5240             0.0       0.0       0.0  
1458813   5711087          0.0       0.0       0.0  
          5711145          0.0       0.0       0.0  
...                        ...       ...       ...  
622090043 5809879          0.0       0.0       0.0  
          5850628          0.0       0.0       0.0  
622090052 5688691          0.0       0.0       0.0  
622090098 5650609          0.0       0.0       0.0  
622090237 5754853          0.0       0.0       0.0  

[5892244 rows x 8 columns]

In [24]:
preferences.reset_index(inplace=True)
preferences.to_csv('prefs.csv', index=False, header=False)

# Generating item - price CSV file

In [34]:
prices_oct = data_oct[['product_id', 'price']]
prices_nov = data_nov[['product_id', 'price']]
prices_dec = data_dec[['product_id', 'price']]
prices_jan = data_jan[['product_id', 'price']]
prices_feb = data_feb[['product_id', 'price']]
prices = pd.concat([prices_oct, prices_nov, prices_dec, prices_jan, prices_feb], axis=0, ignore_index=True).groupby(['product_id']).mean()

In [37]:
prices.reset_index(inplace=True)
prices.to_csv('items.csv', index=False, header=False)